In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

#importing prophet
from fbprophet import Prophet

import pandas as pd
import numpy as np
from fbprophet import Prophet
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import add_changepoints_to_plot

#Visualizing Interactive Plots
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go
from datetime import datetime

#today_date

# Initialize plotly
init_notebook_mode(connected=True)

%matplotlib inline

#weekend
def weekend(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 5 or date.weekday() == 6:
        return 1
    else:
        return 0

#usHolidays
usHolidays = pd.DataFrame({
  'holiday': 'usHolidays',
  'ds': pd.to_datetime(['2016-01-18', '2016-02-15', '2016-05-30', '2016-07-04',
               '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24',
               '2016-12-26', '2017-01-02', '2017-01-16', '2017-02-20',
               '2017-05-29', '2017-07-04', '2017-09-04', '2017-10-09',
               '2017-11-10', '2017-11-23', '2017-12-25', '2018-01-01',
               '2018-01-15', '2018-02-19', '2018-05-28', '2018-07-04',
               '2018-09-03', '2018-10-08', '2018-11-12', '2018-11-22',
               '2018-12-25', '2019-01-01', '2019-01-21', '2019-02-18',
               '2019-05-27', '2019-07-04', '2019-09-02', '2019-10-14',
               '2019-11-11', '2019-11-28', '2019-12-25','2019-12-31']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays =usHolidays
holidays

#plotly_df
def plotly_df(df, title=''):
    """Visualize all the dataframe columns as line plots."""
    common_kw = dict(x=df.index, mode='lines')
    data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)

# Load dataset
cust_orders = pd.read_csv("master_codes_test.csv", index_col= 'Group')
#fill with 0
cust_orders_head=cust_orders.fillna(0)
#create dataframe
df = pd.DataFrame(cust_orders_head)

df['Date'] =pd.to_datetime(df.Date)
df = df.sort_values('Date')

today_date = datetime.today().strftime('%Y-%m-%d')
mask = (df['Date'] < today_date)
df_mask = df.loc[mask]

df_mask['weekend'] = df_mask['Date'].apply(weekend)
df_mask['weekend']
df_wo_wknd = df_mask[df_mask.weekend != 1]

df_wo_wknd_re = df_wo_wknd
df_wo_wknd_re = df_wo_wknd_re.drop(['weekend'], axis=1)
df_wo_wknd_re = df_wo_wknd_re.reset_index()

df_wo_wknd_re = df_wo_wknd_re.set_index('Date')

#get unique master/group codes
group_code_unique = df_wo_wknd_re['Group'].unique()

#get unique dates
#date_unique = df_wo_wknd_re.index.unique()

#print('date_unique ',date_unique)

master_index=0
#for each master/group code, genrate loop
for master in group_code_unique:
    print(master)
    df_wo_wknd_ele = df_wo_wknd_re.loc[df_wo_wknd_re['Group'] == master]

    df_wo_wknd_ele = df_wo_wknd_ele.reset_index()

    df_wo_wknd_ele = df_wo_wknd_ele.drop(['Group'],axis=1)
    df_wo_wknd_ele_tr = df_wo_wknd_ele.transpose()
    grouped = df_wo_wknd_ele.groupby(['Date'])['Quantity'].apply(np.sum)
    grouped_df =pd.DataFrame(grouped,columns= [ 'Quantity'])
    grouped_df.columns =[master]
    grouped_df_transposed = grouped_df.transpose()
    # Splitting Train test data for individual Master codes
    train_size = int(len(grouped_df) * 0.9)
    test_size = len(grouped_df) - train_size
    train = grouped_df[:train_size]
    test = grouped_df[train_size:len(grouped_df)]
    train['Date'] = train.index
    train['Date'] = train['Date'].astype('datetime64[ns]')
    train = train.set_index('Date')
    test['Date'] = test.index
    test['Date'] = test['Date'].astype('datetime64[ns]')
    test = test.set_index('Date')
    test_transposed = test.transpose()
    test['ds'] = test.index
    test['ds'] = test['ds'].astype('datetime64[ns]')
    train_transposed = train.transpose()
    test_transposed = test.transpose()
    #print('--------------------------------test_transposed--------------------------------')
    #print(test_transposed)
    # Prediction size
    prediction_size = 300
    new = pd.DataFrame(train_transposed.iloc[0])
    #ax = plt.gca()
    
    # create 'ds' and 'y' elements for Prophet ML algo
    new['ds'] = train_transposed.columns
    new['ds'] = new['ds'].astype('datetime64[ns]')
    new.columns = ['y', 'ds']
    new_table = new[['ds','y']]
    new_table.set_index('ds')
    
    #create Model by calling Prophet
    my_model = Prophet(interval_width=0.95, holidays=holidays)
    
    #Add Daily Seasonality
    my_model.add_seasonality(name='daily', period=1, fourier_order=4)
    
    #Plot Graph with existing data
    #plotly_df(new,'Daily Shipment Seasonality')
    
    # Fit model on train data
    my_model.fit(new_table)
    
    # Make Future Dataframe with daily frequency and Prediction size
    future_dates = my_model.make_future_dataframe(periods = prediction_size, freq ="d")
    #print('future_dates ',future_dates)
    #Predict Future on Test data
    forecast = my_model.predict(future_dates)
    forecast = round(forecast)
    
    #Plot Forecast
    #figure = my_model.plot(forecast)
    #for changepoint in my_model.changepoints:
    #    plt.axvline(changepoint,ls='--', lw=1)

    # Merge forecasts with given IDs for Model 1
    test_merge = pd.merge(forecast, test, on=['ds'] , how='outer')
    cols = ['ds','yhat']
    test_new = test_merge[cols]
    test_new.yhat[test_new.yhat < 0] = 0

    # Forecasted data
    fc = test_new[['ds', 'yhat']].copy()
    fc[master] = fc['yhat']
    fc =  fc.drop('yhat', axis=1)
    fc['ds'] = fc['ds'].astype('datetime64[ns]')

    fc_transposed = fc.transpose()
    fc_transposed.columns = fc.index.values
    fc['weekend'] = fc['ds'].apply(weekend)
    fc = fc[fc.weekend != 1]

    fc = fc.drop(['weekend'], axis=1)
    print('fc ',fc)
    print(fc['ds'])
    
    if master_index == 0:
        idx = 0
        columns = train.columns
        fc_re = pd.DataFrame(columns=columns)
        grouped_df_transposed_re = pd.DataFrame(columns=columns)
        new_col = fc_transposed.iloc[master_index] #be a list, a Series, an array or a scalar 
        
        fc_re.insert(loc=idx, column='dates', value=new_col)
        fc_re['ds'+master] = fc_re['dates'].astype('datetime64[ns]')
        #fc_re.insert(loc=idx, column='ds'+master, value=fc['ds'])
        #fc_re['ds'] = fc_re['ds'].astype('datetime64[ns]')
    else:
        fc_re['ds'+master] = fc['ds']
    grouped_df_transposed_re[master] = grouped_df[master]
    
    
    fc_re[master] = fc[master]
    
    #print(fc_re[master])

    #fc_re[master] = fc_re[master].fillna(0)
    #fc_re['weekend'+master] = fc_re['ds'+master].apply(weekend)
    #fc_re = fc_re[fc_re['weekend'+master] != 1]

    test_new = round(test_new)
    test_new_df = test_new[['ds', 'yhat']].copy()
    test_new_df['y']=test_new['yhat'] #copy the log-transformed data to another column
    
    #my_model.plot_components(forecast)

    #fig = my_model.plot(forecast)
    #a = add_changepoints_to_plot(fig.gca(), my_model, forecast)
    master_index+=1
    print('------------starts----------')
    #print('fc_re inside loop ', fc_re)
    print(fc_re['ds'+master])
    #fc_re = fc_re[fc_re['ds'+master] < pd.datetime(2020, 1, 1)]
    print('------------ends----------')
print('Forecasting done')
#print('fc_re ', fc_re)
print('-------------grouped_df_transposed_re--------------------')
print(grouped_df_transposed_re)

#for master in group_code_unique:
#    fc_re['weekend'] = fc_re['ds'+master].apply(weekend)
#    fc_re = fc_re[fc_re.weekend != 1]
#to add later on
#fc_re['weekend'] = fc_re['ds'].apply(weekend)
#fc_re
#fc_re = fc_re[fc_re.weekend != 1]
#fc_re
fc_re

In [ ]:
for master in group_code_unique:
    fc_re = fc_re[fc_re['ds'+master] < pd.datetime(2020, 1, 1)]

In [ ]:
#fc_re[(fc_re['dates'].dt.date <'2020-01-01') ]
fc_re = fc_re[fc_re['dates'] < pd.datetime(2020, 1, 1)]

In [ ]:
fc_re

In [ ]:
writer = ExcelWriter('fc_re_forecast_re_with_weekend.xlsx')
fc_re.transpose().to_excel(writer,'Sheet1',index=True)

writer.save()

In [2]:
from datetime import timedelta, date
import pandas as pd

start_date = date(2017, 1, 3)
end_date = date(2020, 1, 3)
dataframeall = pd.DataFrame()
dataframeall = pd.concat([pd.DataFrame({'Date': pd.date_range(start_date, end_date, freq='d')}, columns=['Date'])])
dataframeall

dataframeall['Date']

0      2017-01-03
1      2017-01-04
2      2017-01-05
3      2017-01-06
4      2017-01-07
5      2017-01-08
6      2017-01-09
7      2017-01-10
8      2017-01-11
9      2017-01-12
10     2017-01-13
11     2017-01-14
12     2017-01-15
13     2017-01-16
14     2017-01-17
15     2017-01-18
16     2017-01-19
17     2017-01-20
18     2017-01-21
19     2017-01-22
20     2017-01-23
21     2017-01-24
22     2017-01-25
23     2017-01-26
24     2017-01-27
25     2017-01-28
26     2017-01-29
27     2017-01-30
28     2017-01-31
29     2017-02-01
          ...    
1066   2019-12-05
1067   2019-12-06
1068   2019-12-07
1069   2019-12-08
1070   2019-12-09
1071   2019-12-10
1072   2019-12-11
1073   2019-12-12
1074   2019-12-13
1075   2019-12-14
1076   2019-12-15
1077   2019-12-16
1078   2019-12-17
1079   2019-12-18
1080   2019-12-19
1081   2019-12-20
1082   2019-12-21
1083   2019-12-22
1084   2019-12-23
1085   2019-12-24
1086   2019-12-25
1087   2019-12-26
1088   2019-12-27
1089   2019-12-28
1090   201

In [ ]:
#fc_re['Dates'] = dataframeall['Date']

In [ ]:
dataframeall['weekend'] = dataframeall['Date'].apply(weekend)
dataframeall = dataframeall[dataframeall.weekend != 1]
dataframeall
dataframeall = dataframeall.drop(['weekend'],axis=1)
dataframeall
dataframeall.index = range(0,len(dataframeall ))
dataframeall

In [ ]:
for master in group_code_unique:
    fc_re['weekend'] = fc_re['ds'+master].apply(weekend)
    fc_re
    fc_re = fc_re[fc_re.weekend != 1]
fc_re
#fc_re.index = range(0,len(fc_re ))
#fc_re

In [ ]:
for master in group_code_unique:
    fc_re['weekend'] = fc_re['ds'+master].apply(weekend)
    fc_re
    fc_re = fc_re[fc_re.weekend != 1]
fc_re

In [ ]:
sdfsdf

In [ ]:
fc_re['dsM100915'] = fc_re['dsM100915'].fillna('1990-12-12').astype('datetime64[ns]')
fc_re

In [ ]:
fc_re

In [ ]:
#if fc_re[fc_re.dsM100915.isnull()]:
#    fc_re.dsM100915 = 0
#print(fc_re)
fc_re[pd.notnull(fc_re['dsM100915'])]

In [ ]:
import numpy
#fc_re['DateAll'] =  dataframeall
#fc_re['DateAll']
#fc_re[fc_re['DateAll'].notnull()]
fc_re
#print(dataframeall['Date'].dt.date)
#fc_re['dsMD80105'] = fc_re['dsMD80105'].astype('datetime64[ns]').dt.date

dates = pd.to_datetime(pd.Series(dataframeall['Date']), format = '%Y%m%d')
dates = dates.apply(lambda x: x.strftime('%Y-%m-%d'))


#rint('dates ',dates)
#print(dates.dtype)
print(len(group_code_unique))
new_re = pd.DataFrame()
for k in range(len(group_code_unique)):
    #print(k)
    fc_re['ds'+group_code_unique[k]] = fc_re['ds'+group_code_unique[k]].fillna('1990-12-12').astype('datetime64[ns]')
    fc_redates = pd.to_datetime(pd.Series(fc_re['ds'+group_code_unique[k]]))
    fc_redates = fc_redates.apply(lambda x: x.strftime('%Y-%m-%d'))
    #rint(fc_redates)
    #print(fc_re['ds'+group_code_unique[k]])
    
    #new_df_fc_redates['Date'] = dataframeall['Date']
    i=0
    new_df_fc_redates = pd.DataFrame(columns =['Date',group_code_unique[k]])
    for single in dates:
        #print(i)
        #print(single)
        #print('yo')
        vals = fc_redates[fc_redates.astype(str).str.contains(single)]
        #print(vals)
        if (len(vals) == 1):
            vals =  pd.to_datetime(vals.values)
            value = fc_re.loc[i,group_code_unique[k]]
            #print(value)
            #new_df_fc_redates.loc[i] = [vals.values.to_pydatetime()] + value
            new_df_fc_redates = new_df_fc_redates.append({'Date':vals.values, group_code_unique[k]:value}, ignore_index=True)
            #int(new_df_fc_redates)
        else:
            #print('else')
            i=i-1
            vals =  pd.to_datetime(vals.values)
            #print('else vals',vals)
            #value = fc_re.loc[i,group_code_unique[0]]
            #value = fc_re.loc[i,group_code_unique[0]]
            #print(value)
            new_df_fc_redates = new_df_fc_redates.append({'Date':vals.values, group_code_unique[k]:0}, ignore_index=True)
            #new_df_fc_redates.loc[i] = [0] + [0]
            #rint(new_df_fc_redates)
        i+=1
    #rint('start')
    #rint(new_df_fc_redates)
    print(new_df_fc_redates)
    new_re[group_code_unique[k]] = new_df_fc_redates[group_code_unique[k]]
    #rint('yo')
    #rint(new_re[group_code_unique[k]])
    #ew_df_fc_redates+=new_df_fc_redates
    #rint(new_re)
    #rint('end')
new_df_fc_redates

In [ ]:
new_re

In [ ]:
new_re['ds'] = dataframeall

In [ ]:
new_re

In [ ]:
#grouped_df_transposed_re
writer = ExcelWriter('new_re.xlsx')
new_re.transpose().to_excel(writer,'Sheet1',index=True)
writer.save()

In [ ]:
vals = dates[dates.astype(str).str.contains(single)]

In [ ]:
fc_re.head(50)

In [ ]:
fc_re.loc[fc_re['ds'+master].isin(dataframeall['Date'])]

In [ ]:
#grouped_df_transposed_re
writer = ExcelWriter('grouped_df_transposed_re.xlsx')
grouped_df_transposed_re.transpose().to_excel(writer,'Sheet1',index=True)
writer.save()

In [ ]:
fc_re['weekend'] = fc_re['ds'].apply(weekend)
fc_re = fc_re[fc_re.weekend != 1]
output = fc_re.transpose()
output.columns = fc_re['ds']
new_header = output.iloc[0] #grab the first row for the header
reoutput = output[1:] #take the data less the header row
reoutput.columns = new_header #set the header row as the df header
reoutput = reoutput.drop(['ds'],axis = 0)
reoutput = reoutput.astype('float64')
grouped_df_transposed.columns = pd.to_datetime(grouped_df_transposed.columns)
new_values =reoutput.copy()
reoutput.update(grouped_df_transposed_re.transpose())
reoutput = reoutput.drop(['weekend'],axis = 0)
new_values = new_values.drop(['weekend'],axis = 0)
vals = grouped_df_transposed_re.iloc[:,0:]
vals.to_dict('series')
dataframe = pd.DataFrame(new_values)
diff = dataframe.subtract(vals.transpose(), axis = 1)
diff =abs(diff)
# Absolute Values - sum over the column axis. 
sum_absolute_vals = diff.sum(axis = 1, skipna = True)
sum_absolute_vals
diff
new_values_trans = new_values.transpose()
new_values_trans

In [ ]:
#new_values_trans.set_index('Date', inplace=True)
new_values_trans.index = pd.to_datetime(new_values_trans.index)
new_values_trans = new_values_trans.resample('1M').sum()
new_values_trans

In [ ]:
import calendar
new_values_trans['month'] = pd.to_datetime(new_values_trans.index).month
new_values_trans['year'] = pd.to_datetime(new_values_trans.index).year
new_values_trans['month'] 
new_values_trans['month'] = new_values_trans['month'].apply(lambda x: calendar.month_abbr[x])
new_values_trans
#pd.merge(new_values_trans['month'], new_values_trans['year'], on=new_values_trans.index, how='inner')
#new_values_trans['month'] + new_values_trans['year']
new_values_trans['month'] = new_values_trans['month'].astype(str)
new_values_trans['year'] = new_values_trans['year'].astype(str)
new_values_trans['Period'] = new_values_trans['month']+ ', ' + new_values_trans['year']
new_values_trans['Period']
new_values_trans
new_values_trans = new_values_trans.drop(['month','year'], axis=1)
new_values_trans = new_values_trans.reset_index()
new_values_trans = new_values_trans.set_index('Period')
new_values_trans
new_values_trans = new_values_trans.drop('dates',axis=1)
new_values_trans
#new_values_trans['period']
#pd.merge([new_values_trans['month'].dropna(), new_values_trans['year'].dropna()], how='outer')

In [ ]:
#reoutput.insert(0, "Error Rate", mape_error_rate, True) 
#reoutput
writer = ExcelWriter('forecast_cust_orders_test.xlsx')
new_values_trans.transpose().to_excel(writer,'Sheet1',index=True)
writer.save()

In [ ]:
#not required
reoutput.update(new_values_trans.transpose())
writer = ExcelWriter('forecast_cust_orders_test_old_n_new.xlsx')
reoutput.to_excel(writer,'Sheet1',index=True)
writer.save()

In [ ]:
#not required

reoutput.update(vals.transpose())

#Actual Values
sum_actual_vals = reoutput.sum(axis = 1, skipna = True)
sum_actual_vals

#MAPE Calculation
mape_error_rate = sum_absolute_vals.div(sum_actual_vals)
mape_error_rate

reoutput.insert(0, "Error Rate", mape_error_rate, True) 
writer = ExcelWriter('Customer_orders_Prediction_new_1172019_change.xlsx')
reoutput.to_excel(writer,'Sheet1',index=True)
writer.save()

#new_values.insert(0, "Error Rate", mape_error_rate, True) 
#new_values
#writer = ExcelWriter('Customer_orders_Prediction_wo_old_new_1172019_test.xlsx')
#new_values.to_excel(writer,'Sheet1',index=True)
#writer.save()